In [1]:
!wget -O /tmp/Nightlights_2003_tif.zip "http://ede.grid.unep.ch/download/Nightlights_2003_tif.zip"

!unzip /tmp/Nightlights_2003_tif.zip -d /tmp

--2018-02-07 05:02:19--  http://ede.grid.unep.ch/download/Nightlights_2003_tif.zip
Resolving ede.grid.unep.ch (ede.grid.unep.ch)... 129.194.205.21
Connecting to ede.grid.unep.ch (ede.grid.unep.ch)|129.194.205.21|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13300951 (13M) [application/zip]
Saving to: ‘/tmp/Nightlights_2003_tif.zip’

/tmp/Nightlights_20 100%[===================>]  12.68M  20.3MB/s    in 0.6s    

2018-02-07 05:02:20 (20.3 MB/s) - ‘/tmp/Nightlights_2003_tif.zip’ saved [13300951/13300951]

Archive:  /tmp/Nightlights_2003_tif.zip
  inflating: /tmp/Nighlights 2003.lyr  
  inflating: /tmp/Nightlights_2003.tif  
  inflating: /tmp/Nightlights_2003.xml  


In [2]:
!gdalinfo /tmp/Nightlights_2003.tif

Driver: GTiff/GeoTIFF
Files: /tmp/Nightlights_2003.tif
Size is 43201, 15601
Coordinate System is:
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0],
    UNIT["degree",0.0174532925199433],
    AUTHORITY["EPSG","4326"]]
Origin = (-180.000000000000000,65.000000000000000)
Pixel Size = (0.008333330042660,-0.008333330042660)
Metadata:
  AREA_OR_POINT=Area
  TIFFTAG_RESOLUTIONUNIT=1 (unitless)
  TIFFTAG_SOFTWARE=IMAGINE TIFF Support
Copyright 1991 - 1999 by ERDAS, Inc. All Rights Reserved
@(#)$RCSfile: etif.c $ $Revision: 1.10.1.9.1.9.2.11 $ $Date: 2004/09/15 18:42:01EDT $
  TIFFTAG_XRESOLUTION=1
  TIFFTAG_YRESOLUTION=1
Image Structure Metadata:
  INTERLEAVE=BAND
Corner Coordinates:
Upper Left  (-180.0000000,  65.0000000) (180d 0' 0.00"W, 65d 0' 0.00"N)
Lower Left  (-180.0000000, -65.0082820) (180d 0' 0.00"W, 65d 0'29.82"S)
Upper Right ( 180.0081912,  65.00

In [3]:
!psql -c "SELECT ST_Extent(ST_Transform(geom, 4326)) FROM data.district_area" osm

                                st_extent                                 
--------------------------------------------------------------------------
 BOX(-5.1413332610739 41.3336317564725,9.55996004666832 51.0890003105061)
(1 row)



In [4]:
!gdal_translate \
   -projwin  -5.1413332610739 51.0890003105061  9.55996004666832 41.3336317564725 \
   /tmp/Nightlights_2003.tif /tmp/nightlight_2003_france.tif
    
!gdalwarp -t_srs EPSG:2154 /tmp/nightlight_2003_france.tif /tmp/nightlight_2003_france_2154.tif

!gdalinfo /tmp/nightlight_2003_france_2154.tif

Input file size is 43201, 15601
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 1668P x 1508L.
Processing input file /tmp/nightlight_2003_france.tif.
0...10...20...30...40...50...60...70...80...90...100 - done.
Driver: GTiff/GeoTIFF
Files: /tmp/nightlight_2003_france_2154.tif
Size is 1668, 1508
Coordinate System is:
PROJCS["RGF93 / Lambert-93",
    GEOGCS["RGF93",
        DATUM["Reseau_Geodesique_Francais_1993",
            SPHEROID["GRS 1980",6378137,298.257222101,
                AUTHORITY["EPSG","7019"]],
            TOWGS84[0,0,0,0,0,0,0],
            AUTHORITY["EPSG","6171"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4171"]],
    PROJECTION["Lambert_Conformal_Conic_2SP"],
    PARAMETER["standard_parallel_1",49],
    PARAMETER["standard_parallel_2",44],
    PARAMETER["latitude_of_origin",46.5],
    PARAMETER

In [5]:
!raster2pgsql -d -s 4326 -I -t auto -Y -C -r /tmp/nightlight_2003_france_2154.tif data.nightlight | psql osm

Processing 1/1: /tmp/nightlight_2003_france_2154.tif
INFO: Using computed tile size: 98x52
BEGIN
DROP TABLE
CREATE TABLE
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 11
COPY 5
CREATE INDEX
ANALYZE
NOTICE:  Adding SRID constraint
NOTICE:  Adding scale-X constraint
NOTICE:  Adding scale-Y constraint
NOTICE:  Adding blocksize-X constraint
NOTICE:  Adding blocksize-Y constraint
NOTICE:  Adding alignment constraint
NOTICE:  Adding coverage tile constraint required for regular blocking
NOTICE:  Adding spatially unique constraint required for regular blocking
NOTICE:  Adding number of bands constraint
NOTICE:  Adding pixel type constraint
NOTICE:  Adding nodata value con